In [15]:
# read sqlite3 database 
import sqlite3
from pathlib import Path
db_path = "../rotowire_100.db"
conn = sqlite3.connect(db_path)
c = conn.cursor()
query = "SELECT game_id, report FROM reports"
data = c.execute(query)
# create a folder "reports" to store the reports
Path("../reports_100").mkdir(parents=True, exist_ok=True)
# the reports are named as "game_id.txt"
for d in data:
    game_id = d[0]
    report = d[1]
    with open(f"../reports_100/{game_id}.txt", "w") as f:
        f.write(report)
    
conn.close()

# drop the reports table
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute("DROP TABLE reports")
# create a new table "reports" with columns "game_id" and "report"
query = "CREATE TABLE reports (game_id TEXT, report_url TEXT)"
c.execute(query)
# insert the reports_url into the new table
reports_path = Path("../reports_100")
report_paths = list(reports_path.iterdir())
report_paths.sort(key=lambda x: int(x.stem))
for report_path in report_paths:
    game_id = report_path.stem
    report_url = f"reports/{game_id}.txt"
    c.execute("INSERT INTO reports (game_id, report_url) VALUES (?, ?)", (game_id, report_url))
conn.commit()
conn.close()